[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/khetansarvesh/Tabular-Cross-Sectional-Modelling/blob/main/modelling/data-augmentation/Undercomplete-Autoencoder.ipynb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf
tf.reset_default_graph() 
from tensorflow.contrib.layers import fully_connected

In [ ]:
# reading dataset
df = pd.read_csv('anonymized_data.csv')

# scaling the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop('Label',axis=1))

In [ ]:
# defining network params
num_inputs = 30
neurons_hid1 = 15
neurons_hid2 = 10
neurons_hid3 = neurons_hid1
num_outputs = num_inputs
learning_rate = 0.01
num_epochs = 5
batch_size = 150

# weihts and bias
w1 = tf.Variable(initializer([num_inputs, neurons_hid1]), dtype=tf.float32)
b1 = tf.Variable(tf.zeros(neurons_hid1))
w2 = tf.Variable(initializer([neurons_hid1, neurons_hid2]), dtype=tf.float32)
b2 = tf.Variable(tf.zeros(neurons_hid2))
w3 = tf.Variable(initializer([neurons_hid2, neurons_hid3]), dtype=tf.float32)
b3 = tf.Variable(tf.zeros(neurons_hid3))
w4 = tf.Variable(initializer([neurons_hid3, num_outputs]), dtype=tf.float32)
b4 = tf.Variable(tf.zeros(num_outputs))

In [ ]:
# creating placeholder for the network
X = tf.placeholder(tf.float32, shape=[None, num_inputs])

In [ ]:
initializer = tf.variance_scaling_initializer()

In [ ]:
# DL architecture with 3 hidden layer, H1 as encoder, H2 as lower dim vector and H3 as decoder
hid_layer1 = tf.nn.relu(tf.matmul(X, w1) + b1)
hid_layer2 = tf.nn.relu(tf.matmul(hid_layer1, w2) + b2)
hid_layer3 = tf.nn.relu(tf.matmul(hid_layer2, w3) + b3)
output_layer = tf.matmul(hid_layer3, w4) + b4

'''if Linear Autoencoder PCA architecture'''
#hidden = fully_connected(X, num_hidden, activation_fn=None)
#outputs = fully_connected(hidden, num_outputs, activation_fn=None)

# defining MSE loss function
loss = tf.reduce_mean(tf.square(output_layer - X))

# using adam optimizer to train the network
optimizer = tf.train.AdamOptimizer(learning_rate) #tf.train.RMSPropOptimizer
train = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)    
    for epoch in range(num_epochs):
        num_batches = X.shape[0] // batch_size        
        for iteration in range(num_batches):
            sess.run(train, feed_dict={X: scaled_data})
        training_loss = loss.eval(feed_dict={X: X_batch})   
        dim_red_features = hid_layer2.eval(feed_dict={X: scaled_data}) # Now ask for the hidden layer output (the 2 dimensional output)
        print("Epoch {} Complete. Training Loss: {}".format(epoch,training_loss))

Epoch 0 Complete. Training Loss: 0.03161703050136566
Epoch 1 Complete. Training Loss: 0.026947470381855965
Epoch 2 Complete. Training Loss: 0.025991328060626984
Epoch 3 Complete. Training Loss: 0.026057124137878418
Epoch 4 Complete. Training Loss: 0.02522892877459526


In [ ]:
dim_red_features.shape